# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [131]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gvts
import geoviews.feature as gf


# Import API key
from api_keys import geoapify_key

In [132]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/mahermubarak/Desktop/Modules /python-api-challenge/WeatherPy/Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,29.89,58,0,0.89,US,1720368008
1,1,bethel,41.3712,-73.4140,30.36,71,80,0.45,US,1720368141
2,2,puerto ayora,-0.7393,-90.3518,24.98,89,29,2.24,EC,1720368296
3,3,port-aux-francais,-49.3500,70.2167,1.33,82,88,12.15,TF,1720368297
4,4,margaret river,-33.9500,115.0667,15.69,58,52,5.08,AU,1720368298


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [133]:
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size= "Humidity",
    scale=1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM"
)
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [134]:
# Narrow down cities that fit criteria and drop any results with null values
new_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
new_df.dropna(inplace=True)

# Reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

new_df


/var/folders/jc/gvg7tfv51l5198j_3byfp91w0000gn/T/ipykernel_2910/4236439894.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,104,guerrero negro,27.9769,-114.0611,24.22,60,0,2.71,MX,1720368410
1,150,portland,45.5234,-122.6762,26.72,56,0,0.89,US,1720368464
2,177,farkhar,36.5729,69.8578,22.07,27,0,1.06,AF,1720369008
3,221,tagab,34.8550,69.6492,24.09,26,0,0.44,AF,1720369059
4,262,xiaodian,39.2519,117.2178,23.00,88,0,3.00,CN,1720369103
5,284,cide,41.8921,33.0044,25.28,69,0,0.26,TR,1720369128
6,333,cadillac,44.2519,-85.4012,23.95,57,0,2.57,US,1720369185
7,345,vernon,50.2581,-119.2691,24.78,36,0,1.54,CA,1720369188
8,350,sturgeon bay,44.8342,-87.3770,24.98,73,0,3.60,US,1720368928
9,412,saint-francois,46.4154,3.9054,22.58,48,0,3.60,FR,1720369273


### Step 3: Create a new DataFrame called `hotel_df`.

In [135]:
# Narrow down cities that fit criteria and drop any results with null values
hotel_df = new_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,guerrero negro,MX,27.9769,-114.0611,60,
1,portland,US,45.5234,-122.6762,56,
2,farkhar,AF,36.5729,69.8578,27,
3,tagab,AF,34.8550,69.6492,26,
4,xiaodian,CN,39.2519,117.2178,88,
5,cide,TR,41.8921,33.0044,69,
6,cadillac,US,44.2519,-85.4012,57,
7,vernon,CA,50.2581,-119.2691,36,
8,sturgeon bay,US,44.8342,-87.3770,73,
9,saint-francois,FR,46.4154,3.9054,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [156]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "type": "accommodation.hotel",
    "apiKey": "276a986b773f4817866d775b1090194f",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lon

    # Set base URL for searching hotels
    base_url = "https://api.geoapify.com/v2/places"
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
portland - nearest hotel: the Hoxton
farkhar - nearest hotel: دنیا هوتل
tagab - nearest hotel: صالون عروسی پروان سبز
xiaodian - nearest hotel: 维多利亚国际大酒店
cide - nearest hotel: Pinar Otel
cadillac - nearest hotel: Holiday Inn
vernon - nearest hotel: Okanagan Royal Park Inn
sturgeon bay - nearest hotel: Stone Harbor
saint-francois - nearest hotel: Chez Lily
hillsboro - nearest hotel: Residence Inn Portland Hillsboro/Brookwood
dawson creek - nearest hotel: Comfort Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
0,guerrero negro,MX,27.9769,-114.0611,60,Plaza sal paraiso
1,portland,US,45.5234,-122.6762,56,the Hoxton
2,farkhar,AF,36.5729,69.8578,27,دنیا هوتل
3,tagab,AF,34.8550,69.6492,26,صالون عروسی پروان سبز
4,xiaodian,CN,39.2519,117.2178,88,维多利亚国际大酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [1]:
%%capture --no-display

# Configure the map plot
new_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size= "Humidity",
    scale=1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country", "Humidity", "City"]
    )

# Display the map
new_map

NameError: name 'hotel_df' is not defined